In [1]:
import pandas as pd
import os
import datetime
import locale
import re
import numpy
# import keras
#import tensorflow as tf
#import sklearn.preprocessing as pp
import matplotlib.pyplot as plt 

#from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# 교통량 데이터 로드
traffic = pd.read_csv('traffic_adj.csv')

In [3]:
# 추정유동인구 데이터 로드 
# `17~`19년 3개 파일
flt17 = pd.read_csv('FLT_2017.csv')
flt18 = pd.read_csv('FLT_2018.csv')
flt19 = pd.read_csv('FLT_2019.csv')

In [4]:
# 날씨 데이터 로드
# `17~`19년 3개 파일
nalc17 = pd.read_csv('weather2017.csv', encoding = 'euc-kr')
nalc18 = pd.read_csv('weather2018.csv', encoding = 'euc-kr')
nalc19 = pd.read_csv('weather2019.csv', encoding = 'euc-kr')

# 1개 변수로 병합
weather = pd.concat([nalc17, nalc18, nalc19], axis=0).reset_index()

In [5]:
# 날씨 데이터에서 필요한 컬럼만 추출
weather2 = weather[['일시','시정(10m)', '현상번호(국내식)']]

# 조회 편의를 위한 컬럼명 수정
weather2.columns = ['일시','시정','날씨']

# 일시 컬럼을 이용해 merge를 위한 key 컬럼인 날짜와 시간을 만듬
weather2['날짜'] = weather['일시'].apply(lambda x: str(x)[:10].replace('-',''))
weather2['시간'] = weather['일시'].apply(lambda x: str(x)[-5:-3].replace(' ',''))

# 이후 일시 컬럼 제거
weather2 = weather2.drop('일시', axis=1)

<ipython-input-5-c0eb0f6e10fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather2['날짜'] = weather['일시'].apply(lambda x: str(x)[:10].replace('-',''))
<ipython-input-5-c0eb0f6e10fa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather2['시간'] = weather['일시'].apply(lambda x: str(x)[-5:-3].replace(' ',''))


In [6]:
# 날씨 데이터에서 null 값은 맑은 날의 경우임. 맑음을 0으로 설정함 
weather2['날씨'] = weather2['날씨'].fillna(0)

# nalcode 및 날씨 컬럼은 소수점을 없애고 코드로 활용하기 위해 float => int => str로 타입 변환
# 두 변수는 같으나, 헷갈려서 두 개로 선언했음
nalcode = weather2['날씨'].apply(lambda x: str(int(x)))
weather2['날씨'] = weather2['날씨'].apply(lambda x: str(int(x)))

# 날씨 코드는 2자리씩 묶어서 봐야함
# 각 값의 자리수가 2의 배수면 pass
# 그렇지 않으면 맨 앞에 0을 추가하여 2자리 단위 코드를 만듬.
weather2.loc[nalcode[nalcode.apply(len) % 2 == 1].index, '날씨'] = weather2.loc[nalcode[nalcode.apply(len) % 2 == 1].index, '날씨'].apply(lambda x: '0'+x)

In [7]:
# 날씨 데이터를 2자리씩 묶어 리스트로 반환하는 for 문임
# 이후 코드 테이블과 join하여 한글로 나타내기 위함

time_nalc = pd.Series()            # 변수 초기화
for items in weather2.날씨:       # 날씨 데이터의 각 행을 items로 반환
    item = []                      # for문 내 결과값을 반환할 리스트 초기화  
    for idx in range(0,201,2):    # 최대 자리수를 구하기보다 그냥 종료값을 크게 두고 처리했음. idx는 0부터 200까지 2씩 증가함
        if idx+2 <=len(items):    # idx+2 까지가 날씨코드의 길이보다 짧은 경우만 코드 추출
            item.append(items[idx:idx+2])
    time_nalc = time_nalc.append(pd.Series([item]), ignore_index = True)

<ipython-input-7-5ed85bbb2967>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  time_nalc = pd.Series()            # 변수 초기화


In [8]:
# 우리나라 날씨 현상 코드를 추출하여 텍스트로 작성
# 출처는 기상청 보고서임.
text = '''
01) 비 02) 이슬비 03) 착빙성의 비
04) 소낙비 05) 눈 06) 진눈깨비
07) 착빙성의 이슬비08) 소낙눈 09) 소낙성 진눈깨비
10) 싸락눈 11) 가루눈 12) 어는비
13) 싸락우박 14) 우 박 15) 얼 음 침
16) 안개 17) 땅 안 개 18) 얼음안개
19) 박 무 20) 땅 날린눈 21) 높은 날린눈
22) 눈 보 라 23) 이 슬 24) 언 이슬
25) 서 리 26) 서 릿 발 27) 무 빙
28) 수 상 29) 수 빙 30) 조 빙
31) 우 빙 32) 결 빙 33) 용 오 름
34) 해 빙 35) 유 빙 36) 해 명
40) 연 무 41) 먼지연무 42) 황 사
43) 연 기 44) 강 회 45) 땅날린 먼지
46) 높은날린먼지 47) 먼지보라 48) 회오리 바람
50) 햇 무 리 51) 달 무 리 52) 해 코로나
53) 달 코로나 54) 채 운 55) 무 지 개
56) 어 광 57) 비 숍 환 58) 신 기 루
59) 아지랑이 60) 놀
70) 뇌 전 71) 천 둥 72) 번 개
73) 세인트엘모의 불74) 극 광 75) 지 진
80) 신 적 설 81) 적 설 82) 일조없음
83) 일조 < 20% 84) 일조 ≥ 80% 85) 폭 풍
86) 부분강수
90) 맑음 91) 구름조금 92) 흐림
'''

In [9]:
# 위 텍스트에서 각 번호와 내용을 담는 리스트를 반환하기 위한 과정

text.split("\n")                   # 줄 별로 텍스트를 나눈 후
p = re.compile('\d\d\) [^\d]+')    # '숫자 숫자 ) 띄어쓰기 숫자아님' 패턴을 잡았음
text_list = p.findall(text)        # 해당 패턴을 문서에서 찾아 리스트로 반환

word_list = []                     # 위 리스트를 [번호, 내용]으로 분할하여 담을 리스트 초기화
for word in text_list:
    word_list.append(word.replace(" ","").replace(")"," ").strip().split(" "))   # 괄호를 제거하여 '번호 내용' 형태로 가공 후 split함

In [10]:
word_dict = {}                      # 위 [번호, 내용] 리스트를 dict[번호] = 내용 으로 변환할 변수 초기화
for word in word_list:
    word_dict[word[0]] = word[1]    # key = 번호, item = 내용인 dict 
word_dict['83'] = '일조<20%'        # 패턴으로 잡히지않는 내용은 직접 수정
word_dict['84'] = '일조≥80%'
word_dict['00'] = '맑음'
word_dict                           # 결과 확인

{'01': '비',
 '02': '이슬비',
 '03': '착빙성의비',
 '04': '소낙비',
 '05': '눈',
 '06': '진눈깨비',
 '07': '착빙성의이슬비',
 '08': '소낙눈',
 '09': '소낙성진눈깨비',
 '10': '싸락눈',
 '11': '가루눈',
 '12': '어는비',
 '13': '싸락우박',
 '14': '우박',
 '15': '얼음침',
 '16': '안개',
 '17': '땅안개',
 '18': '얼음안개',
 '19': '박무',
 '20': '땅날린눈',
 '21': '높은날린눈',
 '22': '눈보라',
 '23': '이슬',
 '24': '언이슬',
 '25': '서리',
 '26': '서릿발',
 '27': '무빙',
 '28': '수상',
 '29': '수빙',
 '30': '조빙',
 '31': '우빙',
 '32': '결빙',
 '33': '용오름',
 '34': '해빙',
 '35': '유빙',
 '36': '해명',
 '40': '연무',
 '41': '먼지연무',
 '42': '황사',
 '43': '연기',
 '44': '강회',
 '45': '땅날린먼지',
 '46': '높은날린먼지',
 '47': '먼지보라',
 '48': '회오리바람',
 '50': '햇무리',
 '51': '달무리',
 '52': '해코로나',
 '53': '달코로나',
 '54': '채운',
 '55': '무지개',
 '56': '어광',
 '57': '비숍환',
 '58': '신기루',
 '59': '아지랑이',
 '60': '놀',
 '70': '뇌전',
 '71': '천둥',
 '72': '번개',
 '73': '세인트엘모의불',
 '74': '극광',
 '75': '지진',
 '80': '신적설',
 '81': '적설',
 '82': '일조없음',
 '83': '일조<20%',
 '84': '일조≥80%',
 '85': '폭풍',
 '86': '부분강수',
 '90': '맑음',
 '91': '구름조금',

In [11]:
# 다음 함수는 날씨를 ['날씨', '날씨'] 리스트로 선언한 날씨를 추출하여
# 텍스트 형태 '날씨', '날씨' 로 바꾸기 위한 함수

def matching(x):
    answer = []
    for i in x:
        answer.append(word_dict[str(i)])
    answer = ','.join(answer)
    return answer

time_nalc = time_nalc.apply(lambda x: matching(x))

In [12]:
# 날씨 코드의 도메인을 만들기 위해 고유값을 추출하는 과정
# ,를 기준으로 텍스트로 만든 후 ,를 기준으로 나눠 list로 반환
# 이를 set 형태로 바꾸어 겹치는 값을 제거함
hap = ','.join(list(time_nalc.values))
hap = hap.split(',')
len(set(hap))

14

In [13]:
# 위 데이터를 활용하여 날씨별 컬럼을 만들고 0으로 초기화 함.
# 테이블 크기는 날씨 데이터의 행 x 유니크한 날씨의 개수
nalc_col = pd.DataFrame(numpy.zeros((weather2.shape[0],len(set(hap)))), columns = set(hap))
nalc_col.head(1)

,소낙눈,박무,이슬비,맑음,황사,소낙비,안개,싸락우박,연무,비,싸락눈,눈,우박,진눈깨비
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 위에서 초기화한 날씨 테이블에 각 컬럼에 해당하는 값이 있는 경우 1로 바꿈
# 이를 통해 카테고리 변수로 변환함.
for idx, weather in enumerate(time_nalc):
    items= weather.split(',')
    for item in items:
        nalc_col[item].iloc[idx,] = 1

In [15]:
nalc_col

,소낙눈,박무,이슬비,맑음,황사,소낙비,안개,싸락우박,연무,비,싸락눈,눈,우박,진눈깨비
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26275,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26276,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26277,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26278,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# 이후 변환한 데이터는 삭제함 (날씨 컬럼)
weather2 = weather2.drop('날씨', axis=1)

# 만든 날씨 테이블을 원래 날씨 데이터에 합쳐 시간별 시정 + 날씨 데이터셋으로 만듬
weather3 = pd.concat([weather2, nalc_col], axis =1 )

# 컬럼 파악 편의를 위한 순서 조정
weather3 = weather3[['날짜', '시간', '시정', '싸락우박', '우박', '싸락눈', '비', '소낙비', '연무', '이슬비', '소낙눈',
       '박무', '진눈깨비', '안개', '황사', '맑음', '눈']]

# 여기부터 교통량 데이터 전처리

In [17]:
# csv 파일 자체에 있는 index값 제거
traffic = traffic.drop('Unnamed: 0', axis=1)

# 서울시를 제외한 데이터 제거
# 제거해야할 리스트 작성 
drop_list = ['고양시', '과천시', '광명시', '구리시', '김포시', '부천시', '하남시']

# 제거해야 할 인덱스를 담을 리스트 선언
drop_index = []

for idx, i in enumerate(traffic['구']):   # 구 데이터를 조회해서
    if i in drop_list:                     # 제거대상 리스트에 있으면
        drop_index.append(idx)             # 그 행의 index를 저장함

traffic = traffic.drop(drop_index, axis=0).reset_index() # 위의 index를 제거
set(traffic['구']) # 데이터 검증

{'강남구',
 '강동구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구'}

In [18]:
# 필요한 컬럼만 추출한 변수 선언 (필요없는 컬럼 제거)
trf = traffic.drop(['index', '위도','경도','동','구분','지점번호'], axis=1) 

In [19]:
# 테이블 형태 변환을 위해 melt 함수 사용
trf_melt = trf.melt(['구','일자'], var_name = '시간', value_name = 'traffic')

# 이후 키 기준으로 합을 계산한 테이블 활용
trf_grb = trf_melt.groupby(['구','일자','시간']).sum().reset_index()

In [20]:
# join을 위해 key 타입 변경
trf_grb['시간'] = trf_grb['시간'].apply(lambda x: int(x[:-1]))
trf_grb['날짜'] = trf_grb['일자'].apply(int)
trf_grb['traffic'] = trf_grb['traffic'].apply(int)

# 컬럼 순서 조정
trf_grb = trf_grb[['날짜','구','시간','traffic']]

In [21]:
# 최종 테이블 형태로 가공
trf_grb.sort_values(['날짜','구','시간'])

,날짜,구,시간,traffic
0,20170101,강남구,0,27336
11,20170101,강남구,1,27429
16,20170101,강남구,2,18495
17,20170101,강남구,3,15075
18,20170101,강남구,4,13356
...,...,...,...,...
630706,20191231,중랑구,19,9508
630708,20191231,중랑구,20,8824
630709,20191231,중랑구,21,8125
630710,20191231,중랑구,22,7027


In [22]:
flt = pd.concat([flt17,flt18,flt19], axis= 0, ignore_index= True)
flt

,날짜,시간,구,추정유동인구
0,20170101,0,강남구,449000
1,20170101,1,강남구,441924
2,20170101,2,강남구,436549
3,20170101,3,강남구,431175
4,20170101,4,강남구,425355
...,...,...,...,...
656995,20191231,19,중랑구,278903
656996,20191231,20,중랑구,288610
656997,20191231,21,중랑구,297197
656998,20191231,22,중랑구,306178


In [23]:
# 테이블 shape을 통해 오류 검증
# trf_grb에 강북구가 빠져있음
# trf_grb를 기준으로 데이터셋 merge 시작
trf_grb.shape, flt.shape, weather3.shape

((630720, 4), (657000, 4), (26280, 17))

### merge를 위해 데이터 타입 확인 info() 메소드 활용

In [24]:
trf_grb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630720 entries, 0 to 630719
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   날짜       630720 non-null  int64 
 1   구        630720 non-null  object
 2   시간       630720 non-null  int64 
 3   traffic  630720 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 19.2+ MB


In [25]:
flt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657000 entries, 0 to 656999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   날짜      657000 non-null  int64 
 1   시간      657000 non-null  int64 
 2   구       657000 non-null  object
 3   추정유동인구  657000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 20.1+ MB


In [26]:
weather3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26280 entries, 0 to 26279
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      26280 non-null  object 
 1   시간      26280 non-null  object 
 2   시정      26280 non-null  int64  
 3   싸락우박    26280 non-null  float64
 4   우박      26280 non-null  float64
 5   싸락눈     26280 non-null  float64
 6   비       26280 non-null  float64
 7   소낙비     26280 non-null  float64
 8   연무      26280 non-null  float64
 9   이슬비     26280 non-null  float64
 10  소낙눈     26280 non-null  float64
 11  박무      26280 non-null  float64
 12  진눈깨비    26280 non-null  float64
 13  안개      26280 non-null  float64
 14  황사      26280 non-null  float64
 15  맑음      26280 non-null  float64
 16  눈       26280 non-null  float64
dtypes: float64(14), int64(1), object(2)
memory usage: 3.4+ MB


In [27]:
# 날씨 데이터의 join key 타입 변경
weather3['날짜'] = weather3['날짜'].apply(lambda x: int(x))
weather3['시간'] = weather3['시간'].apply(lambda x: int(x))

In [28]:
# trf_grb를 기준으로 merge 날씨 데이터에는 구 정보가 없음
dataset = pd.merge(flt, trf_grb, how ='right', on = ['날짜','시간','구'])
dataset = pd.merge(dataset, weather3, how ='left', on = ['날짜','시간'])
dataset.head(1)

,날짜,시간,구,추정유동인구,traffic,시정,싸락우박,우박,싸락눈,비,소낙비,연무,이슬비,소낙눈,박무,진눈깨비,안개,황사,맑음,눈
0,20170101,0,강남구,449000,27336,491,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# 음주사고 데이터 로드
acid = pd.read_csv('acid_all.csv', encoding = 'euc-kr')

In [30]:
# 사고 일시를 통해 날짜 및 시간 컬럼 추출 (join key)
date = acid['사고일시'].apply(lambda x: x.split(" "))
acid['year'] = date.apply(lambda x: x[0][:-1])
acid['month']  = date.apply(lambda x: f'{x[1][:-1]:0>2}')
acid['day']  = date.apply(lambda x: f'{x[2][:-1]:0>2}')
acid['hour']  = date.apply(lambda x: f'{x[3][:-1]:0>2}')
acid.drop('사고일시', axis = 1, inplace = True)

acid['날짜'] = acid['year'] + acid['month'] + acid['day']
acid = acid.drop(['year','month','day'], axis=1)

In [31]:
for i,j in enumerate(acid['시군구'].apply(lambda x: x.split(" "))):
    try:
        j[2]
    except:
        print(i)

5447
23991
73786


In [32]:
acid = acid.drop([5447,23991,73786])

In [33]:
# 시군구 컬럼을 통해 구 동 컬럼 추출
acid['구']= acid['시군구'].apply(lambda x: x.split(" ")[1])
acid['동']= acid['시군구'].apply(lambda x: x.split(" ")[2])
acid = acid.drop('시군구', axis=1)

In [41]:
# 교통속도 데이터 업로드
velocity = pd.read_csv('velocity.csv')

In [50]:
velocity[0:1]

,일자,링크아이디,요일,도로명,기능유형구분,도심/외곽구분,권역구분,거리,01시,02시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20170101,1000000100,일,새문안로,주간선도로,도심권,종로구,306.0,11802.42,12034.98,...,10379.52,11297.52,11450.52,10939.5,11144.52,10939.5,10456.02,10122.48,10303.02,11221.02


In [ ]:
for i in range(1, velocity.shape[0]):
    velocity.iloc[i,8:] = velocity.iloc[i,8:] * velocity.iloc[i,7]
velocity[0:2]

In [34]:
# target 변수 설정
# 사망 중상 경상 부상 4개 컬럼으로 나눈 타겟 = target4
# 사고건수 타겟 = target_event
target4 = acid.groupby(['날짜','hour', '구']).sum()[['사망자수','중상자수','경상자수','부상신고자수']].reset_index()
target_event = acid.groupby(['날짜','hour', '구']).count()['사고번호'].reset_index()

In [35]:
# 컬럼 명 변경
target_event.columns = ['날짜','시간','구','건수']
target4.columns = ['날짜', '시간', '구', '사망자수', '중상자수', '경상자수', '부상신고자수']

In [36]:
# join key 타입 변경
target_event['날짜'] = target_event['날짜'].apply(int)
target_event['시간'] = target_event['시간'].apply(int)

target4['날짜'] = target4['날짜'].apply(int)
target4['시간'] = target4['시간'].apply(int)

In [37]:
# 각 타겟 변수에 따른 데이터셋 merge
ds_4 = pd.merge(dataset, target4,  how = 'left', on = ['날짜','시간','구'])
dataset4 = pd.merge(dataset, target_event, how = 'left', on = ['날짜','시간','구'])

# 여기까지 데이터셋 결합

# 여기부터 EDA

# NEWS 데이터 merge 및 EDA

In [38]:
# NEWS 데이터 로드
# 각각 뉴스 발행량, 조회지수, 단속뉴스 데이터
alchol = pd.read_csv('alchol.csv')
click = pd.read_excel('click.xlsx')
check = pd.read_csv('check.csv')

FileNotFoundError: [Errno 2] File alchol.csv does not exist: 'alchol.csv'

In [ ]:
# CSV INDEX 제거
alchol.drop('Unnamed: 0', axis=1,inplace=True)

In [ ]:
# 컬럼명 변경
alchol.columns = ['날짜','뉴스발행량']
click.columns = ['날짜','조회지수']

In [ ]:
# Join key 추출
alchol['날짜'] = alchol['날짜'].apply(lambda x: int(x.replace('.','')))
click['날짜'] = click['날짜'].apply(lambda x: int(x.replace('-','')))

In [ ]:
# 각 데이터 merge
ds_5 = pd.merge(ds_4, alchol, on = '날짜')
ds_6 = pd.merge(ds_5, click, on ='날짜')

dataset5 = pd.merge(dataset4, alchol, on = '날짜')
dataset6 = pd.merge(dataset5, click, on ='날짜')

In [ ]:
# 데이터셋 파악 시작
dataset6.shape

In [ ]:
dataset6.info()

In [ ]:
dataset6.describe()

In [ ]:
# max 값 조회 (확인)
click[click['날짜'] == 20190625]

In [ ]:
# 월별 groupby를 위한 컬럼 생성
click['월'] = click['날짜'].apply(lambda x: str(x)[:6])
alchol['월'] = alchol['날짜'].apply(lambda x: str(x)[:6])

# EDA

In [ ]:
alchol2 = alchol.sort_values('뉴스발행량').reset_index().drop(['index','날짜'],axis=1)
plt.scatter(alchol['월'], alchol['뉴스발행량'])

In [ ]:
aa = dataset6.groupby('월').mean()['건수']
bb = alchol.groupby('월').mean()['뉴스발행량']
pd.concat([aa,bb],axis=1).corr()

In [ ]:
ds_6['날짜'] = ds_6['날짜'].apply(str)
alchol['날짜'] = alchol['날짜'].apply(str)

In [ ]:
pd.set_option('display.max_rows', 999)

In [ ]:
ds_6['월'] = ds_6['날짜'].apply(lambda x: str(x)[:6])

In [ ]:
dataset6['월'] = dataset6['날짜'].apply(lambda x: str(x)[:6])

In [ ]:
ds_6 = ds_6.fillna(0)
dataset6 = dataset6.fillna(0)
corr = dataset6.corr()

In [ ]:
def make_heatmap(ds_6):
    ds_grb = ds_6.groupby(['날짜', '구']).sum().reset_index().drop('시간',axis=1)
    for i in list(set(ds_grb['구'])):
        corr = ds_grb[ds_grb['구'] == i].corr()
        plt.rcParams['font.family'] = 'Malgun Gothic'
        sns.heatmap(corr, 
                xticklabels=corr.columns,
                yticklabels=corr.columns)
        plt.show()

# 모델링

In [ ]:
ds_6.to_csv('ds6_all.csv', index=False)
dataset6.to_csv('dataset_all.csv', index=False)